# Prevendo nivel de satisfação dos clientes santander

Problema de negócio:
Identificar o nivel de clientes insatisfeitos no inicio do relacionamento
Trabalho é um problema de classificação para identificar se cliente deixou ou não deixou de ser cliente do banco

Dados estão no kaggle
https://www.kaggle.com/c/santander-customer-satisfaction

### Utilizando Spark

In [1]:
# imports
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
# criando a sessão do spark
# definindo o master como local
# definindo nome como sparkML
spSession = SparkSession.builder.master('local').appName('sparkMl').getOrCreate()

In [3]:
# criando um RDD a partir do arquivo importado usando spark context
santanderRDD = sc.textFile('santander-customer-satisfaction/train.csv')

In [4]:
# separando os dados por virgula
santanderRDD2 = santanderRDD.map(lambda x: x.split(','))

In [5]:
# obtendo o header
header = santanderRDD2.first()

In [6]:
# removendo primeira linha (header)
santanderRDD3 = santanderRDD2.filter(lambda x: "ID" not in x)

In [7]:
# convertendo rdd para dataframe do spark, usando os dados do header como schema
sandanderDF = santanderRDD3.toDF(header)

In [8]:
# na conversão os dados ficaram como string, convertendo para Double
sandanderDF = sandanderDF.select(*(col(c).cast(DoubleType()).alias(c) for c in sandanderDF.columns))

In [9]:
# removendo a coluna ID, cola não será usada 
sandanderDF = sandanderDF.drop('ID')

In [10]:
# colocando os dados em cache para melhorar performance
sandanderDF.cache()

DataFrame[var3: double, var15: double, imp_ent_var16_ult1: double, imp_op_var39_comer_ult1: double, imp_op_var39_comer_ult3: double, imp_op_var40_comer_ult1: double, imp_op_var40_comer_ult3: double, imp_op_var40_efect_ult1: double, imp_op_var40_efect_ult3: double, imp_op_var40_ult1: double, imp_op_var41_comer_ult1: double, imp_op_var41_comer_ult3: double, imp_op_var41_efect_ult1: double, imp_op_var41_efect_ult3: double, imp_op_var41_ult1: double, imp_op_var39_efect_ult1: double, imp_op_var39_efect_ult3: double, imp_op_var39_ult1: double, imp_sal_var16_ult1: double, ind_var1_0: double, ind_var1: double, ind_var2_0: double, ind_var2: double, ind_var5_0: double, ind_var5: double, ind_var6_0: double, ind_var6: double, ind_var8_0: double, ind_var8: double, ind_var12_0: double, ind_var12: double, ind_var13_0: double, ind_var13_corto_0: double, ind_var13_corto: double, ind_var13_largo_0: double, ind_var13_largo: double, ind_var13_medio_0: double, ind_var13_medio: double, ind_var13: double, in

In [11]:
display(sandanderDF.describe().show())

+-------+-------------------+------------------+------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+------------------+-----------------------+-----------------------+-----------------+------------------+--------------------+--------------------+----------+--------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------

None

In [12]:
# exibindo primeira linha
sandanderDF.head(1)

[Row(var3=2.0, var15=23.0, imp_ent_var16_ult1=0.0, imp_op_var39_comer_ult1=0.0, imp_op_var39_comer_ult3=0.0, imp_op_var40_comer_ult1=0.0, imp_op_var40_comer_ult3=0.0, imp_op_var40_efect_ult1=0.0, imp_op_var40_efect_ult3=0.0, imp_op_var40_ult1=0.0, imp_op_var41_comer_ult1=0.0, imp_op_var41_comer_ult3=0.0, imp_op_var41_efect_ult1=0.0, imp_op_var41_efect_ult3=0.0, imp_op_var41_ult1=0.0, imp_op_var39_efect_ult1=0.0, imp_op_var39_efect_ult3=0.0, imp_op_var39_ult1=0.0, imp_sal_var16_ult1=0.0, ind_var1_0=0.0, ind_var1=0.0, ind_var2_0=0.0, ind_var2=0.0, ind_var5_0=1.0, ind_var5=0.0, ind_var6_0=0.0, ind_var6=0.0, ind_var8_0=0.0, ind_var8=0.0, ind_var12_0=0.0, ind_var12=0.0, ind_var13_0=0.0, ind_var13_corto_0=0.0, ind_var13_corto=0.0, ind_var13_largo_0=0.0, ind_var13_largo=0.0, ind_var13_medio_0=0.0, ind_var13_medio=0.0, ind_var13=0.0, ind_var14_0=0.0, ind_var14=0.0, ind_var17_0=0.0, ind_var17=0.0, ind_var18_0=0.0, ind_var18=0.0, ind_var19=0.0, ind_var20_0=0.0, ind_var20=0.0, ind_var24_0=0.0, in

In [13]:
# Encontrando a correlação entre a variável target com as variáveis preditoras
for i in sandanderDF.columns:
    if not(isinstance(sandanderDF.select(i).take(1)[0][0], str)) :
        print("Correlação da variável TARGET com", i, sandanderDF.stat.corr('TARGET', i))

Correlação da variável TARGET com var3 0.004474798175536685
Correlação da variável TARGET com var15 0.1013220985607169
Correlação da variável TARGET com imp_ent_var16_ult1 -1.7460253766297183e-05
Correlação da variável TARGET com imp_op_var39_comer_ult1 0.010353129575421496
Correlação da variável TARGET com imp_op_var39_comer_ult3 0.0035169224416893973
Correlação da variável TARGET com imp_op_var40_comer_ult1 0.0031193869489658296
Correlação da variável TARGET com imp_op_var40_comer_ult3 -0.0003578690699807045
Correlação da variável TARGET com imp_op_var40_efect_ult1 0.01922144928852695
Correlação da variável TARGET com imp_op_var40_efect_ult3 0.019965206754799267
Correlação da variável TARGET com imp_op_var40_ult1 0.0030872912147492
Correlação da variável TARGET com imp_op_var41_comer_ult1 0.010082404008577805
Correlação da variável TARGET com imp_op_var41_comer_ult3 0.0038585869783322783
Correlação da variável TARGET com imp_op_var41_efect_ult1 0.02947851812252822
Correlação da variá

Correlação da variável TARGET com num_op_var41_hace3 -0.0011949275362012248
Correlação da variável TARGET com num_op_var41_ult1 0.011241645053677312
Correlação da variável TARGET com num_op_var41_ult3 0.010461674068047194
Correlação da variável TARGET com num_op_var39_hace2 0.007185348347176049
Correlação da variável TARGET com num_op_var39_hace3 -0.001351523393070222
Correlação da variável TARGET com num_op_var39_ult1 0.011246031183984552
Correlação da variável TARGET com num_op_var39_ult3 0.010270906024646568
Correlação da variável TARGET com num_var27_0 nan
Correlação da variável TARGET com num_var28_0 nan
Correlação da variável TARGET com num_var28 nan
Correlação da variável TARGET com num_var27 nan
Correlação da variável TARGET com num_var29_0 -0.0020823115452682393
Correlação da variável TARGET com num_var29 -0.001041114683300633
Correlação da variável TARGET com num_var30_0 -0.033902726457946813
Correlação da variável TARGET com num_var30 -0.1382890341310544
Correlação da variáv

Correlação da variável TARGET com imp_trasp_var17_in_hace3 -0.0009801568973627926
Correlação da variável TARGET com imp_trasp_var17_in_ult1 -0.0010020522439791854
Correlação da variável TARGET com imp_trasp_var17_out_hace3 nan
Correlação da variável TARGET com imp_trasp_var17_out_ult1 -0.0010867719583534296
Correlação da variável TARGET com imp_trasp_var33_in_hace3 -0.001748708558855794
Correlação da variável TARGET com imp_trasp_var33_in_ult1 -0.0011960136398964203
Correlação da variável TARGET com imp_trasp_var33_out_hace3 nan
Correlação da variável TARGET com imp_trasp_var33_out_ult1 -0.000736174410464798
Correlação da variável TARGET com imp_venta_var44_hace3 -0.0009467000929316941
Correlação da variável TARGET com imp_venta_var44_ult1 -0.001465033284778904
Correlação da variável TARGET com ind_var7_emit_ult1 -0.0012751082558142494
Correlação da variável TARGET com ind_var7_recib_ult1 0.0011560337613095246
Correlação da variável TARGET com ind_var10_ult1 0.010328785675873383
Correl

Correlação da variável TARGET com saldo_medio_var13_medio_ult1 -0.0009166686799414274
Correlação da variável TARGET com saldo_medio_var13_medio_ult3 -0.0009520629897784518
Correlação da variável TARGET com saldo_medio_var17_hace2 -0.001202255444284875
Correlação da variável TARGET com saldo_medio_var17_hace3 -0.0008593602909916307
Correlação da variável TARGET com saldo_medio_var17_ult1 -0.0005825814801326777
Correlação da variável TARGET com saldo_medio_var17_ult3 -0.00032758746581768673
Correlação da variável TARGET com saldo_medio_var29_hace2 -0.0010341425774526717
Correlação da variável TARGET com saldo_medio_var29_hace3 -0.0007361744104648011
Correlação da variável TARGET com saldo_medio_var29_ult1 -0.000989596914522271
Correlação da variável TARGET com saldo_medio_var29_ult3 -0.001188270554947012
Correlação da variável TARGET com saldo_medio_var33_hace2 -0.0035332898128533495
Correlação da variável TARGET com saldo_medio_var33_hace3 -0.0024314911720265775
Correlação da variável T

In [14]:
# mutando o tipo da variavel target para integer (tipo foi alterado durante o pré-processamento)
sandanderDF = sandanderDF.withColumn("TARGET",sandanderDF.TARGET.cast(IntegerType()))

In [15]:
# Criando um LabeledPoint (target, Vector[features])
def transformaVar(row) :
    obj = (row["TARGET"], Vectors.dense([row["var38"], 
                                          row["saldo_medio_var13_largo_ult3"], 
                                          row["saldo_medio_var13_largo_ult1"], 
                                          row["saldo_medio_var13_largo_hace2"],
                                          row["saldo_medio_var13_corto_ult3"], row["saldo_medio_var13_corto_ult1"],
                                          row["saldo_medio_var13_corto_hace3"], row["saldo_medio_var13_corto_hace2"], 
                                          row["saldo_medio_var12_ult3"], row["saldo_medio_var12_ult1"], row["saldo_medio_var12_hace3"],  
                                          row["saldo_medio_var12_hace2"], row["saldo_medio_var5_ult3"], row["saldo_medio_var5_ult1"], 
                                          row["saldo_medio_var5_hace3"], row["saldo_medio_var5_hace2"], row["num_var43_recib_ult1"],
                                          row["num_reemb_var17_ult1"], row["num_op_var39_efect_ult3"], row["num_op_var39_efect_ult1"],
                                          row["num_op_var41_efect_ult3"], row["num_op_var41_efect_ult1"], row["num_op_var40_efect_ult3"],
                                          row["num_op_var40_efect_ult1"], row["num_meses_var39_vig_ult3"], row["num_meses_var13_largo_ult3"],
                                          row["num_meses_var13_corto_ult3"], row["num_meses_var12_ult3"], row["num_meses_var8_ult3"],
                                          row["num_meses_var5_ult3"], row["num_var22_ult3"], row["num_var22_ult1"], row["ind_var10_ult1"],
                                          row["var36"], row["saldo_var24"], row["saldo_var13"], row["saldo_var13_largo"],
                                          row["saldo_var13_corto"], row["saldo_var12"], row["saldo_var5"],
                                          row["num_var42"], row["num_var37_med_ult2"], row["num_var35"],
                                          row["num_var30"], row["num_var30_0"], row["num_op_var39_ult3"], row["num_op_var41_ult1"],
                                          row["num_var25"], row["num_var26"], row["num_var24"], row["num_var24_0"],
                                          row["num_var20"], row["num_var20_0"], row["num_var13"], row["num_var13_corto"],
                                          row["num_var13_corto_0"], row["num_var12_0"], row["num_var8"], row["num_var8_0"],
                                          row["num_var5"], row["num_var4"], row["ind_var41_0"], row["ind_var30"],
                                          row["ind_var25"], row["ind_var26"], row["ind_var26_0"], row["ind_var26_cte"],
                                          row["ind_var24"], row["ind_var24_0"], row["ind_var13"], row["ind_var5"], row["imp_op_var39_efect_ult1"],
                                          row["imp_op_var41_ult1"], row["imp_op_var39_ult1"], row["imp_op_var41_ult1"], row["imp_op_var41_ult1"],
                                          row["imp_ent_var16_ult1"],row["var15"]]))
    return obj

In [16]:
# converte o dataframe para RDD, para usar os recursos do SPARK, e com a função map aplica a função transformVar, criando um vetor denso
sandanderRDD4 = sandanderDF.rdd.map(transformaVar)

# convert novamente para dataframe do spark
sandanderDF = spSession.createDataFrame(sandanderRDD4,["label", "features"])

# exibe as primeiras 10 linhas
sandanderDF.select("label","features").show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[39205.17,0.0,0.0...|
|    0|[49278.03,0.0,0.0...|
|    0|[67333.77,0.0,0.0...|
|    0|[64007.97,0.0,0.0...|
|    0|[117310.979016494...|
|    0|[87975.75,0.0,0.0...|
|    0|[94956.66,0.0,0.0...|
|    0|[251638.95,0.0,0....|
|    0|[101962.02,0.0,0....|
|    0|[356463.06,0.0,0....|
+-----+--------------------+
only showing top 10 rows



In [17]:
#Aplicando Redução de Dimensionalidade com PCA
featuresPCA = PCA(k = 20, inputCol = "features", outputCol = "pcaFeatures")
pcaModel = featuresPCA.fit(sandanderDF)
pcaResult = pcaModel.transform(sandanderDF).select("label","features")
pcaResult.show(truncate = False)

+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                                                                                                                                                                                                  |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
# Criando o modelo de classificação Kmeans
kmeans = KMeans(k = 5, seed = 1)
modelo = kmeans.fit(pcaResult)

In [19]:
# Previsões
previsoes = modelo.transform(pcaResult)
previsoes.show()

+-----+--------------------+----------+
|label|            features|prediction|
+-----+--------------------+----------+
|    0|[39205.17,0.0,0.0...|         0|
|    0|[49278.03,0.0,0.0...|         0|
|    0|[67333.77,0.0,0.0...|         0|
|    0|[64007.97,0.0,0.0...|         0|
|    0|[117310.979016494...|         0|
|    0|[87975.75,0.0,0.0...|         0|
|    0|[94956.66,0.0,0.0...|         0|
|    0|[251638.95,0.0,0....|         0|
|    0|[101962.02,0.0,0....|         0|
|    0|[356463.06,0.0,0....|         4|
|    0|[125424.06,0.0,0....|         0|
|    0|[75368.52,0.0,0.0...|         0|
|    0|[122220.09,0.0,0....|         0|
|    0|[118436.97,0.0,0....|         0|
|    0|[95294.13,0.0,0.0...|         0|
|    0|[104403.87,0.0,0....|         0|
|    0|[117310.979016494...|         0|
|    0|[153014.13,0.0,0....|         0|
|    0|[168943.23,0.0,0....|         0|
|    0|[96601.14,0.0,0.0...|         0|
+-----+--------------------+----------+
only showing top 20 rows



In [20]:
# Avaliando a acurácia
previsoes = previsoes.withColumn("prediction",previsoes.prediction.cast(DoubleType()))
evaluator = MulticlassClassificationEvaluator(predictionCol = "prediction", labelCol = "label", metricName = "accuracy")
evaluator.evaluate(previsoes) 

0.9138253091291765